In [74]:
import pandas
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 

In [75]:
nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Wojciech\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Wojciech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [76]:
def clean(text):
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(text)])
    return text

In [77]:
data_frame = pandas.read_csv('train.csv', dtype ={'Id': 'object', 'Category': 'object', 'Tweet': 'object'}).dropna()
data = data_frame['Tweet'].map(clean).tolist()
data_labels = data_frame['Category'].tolist()

In [78]:
cv = CountVectorizer()
features = cv.fit_transform(data).toarray()
X_train, X_test, y_train, y_test  = train_test_split(
        features, 
        data_labels,
        train_size=0.80, 
        random_state=1234)

In [79]:
model = MultinomialNB()
model = model.fit(X=X_train, y=y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.559463986599665


In [80]:
test_df = pandas.read_csv('test.csv', dtype ={'Id': 'object', 'Tweet': 'object'}).dropna()
tweets = test_df['Tweet'].map(clean).tolist()
features = cv.transform(tweets).toarray()
predictions = model.predict(features)
rows=[]
for index,row in test_df.iterrows():
    rows.append((row['Id'], predictions[index]))
result = pandas.DataFrame(rows, columns = ['Id','Category'])
result.to_csv('result.csv', index=False)
result

,Id,Category
0,628949369883000832,positive
1,628976607420645377,neutral
2,629023169169518592,positive
3,629179223232479232,positive
4,629186282179153920,neutral
...,...,...
3995,641411385700712448,positive
3996,641452712098406400,positive
3997,635369700298498048,neutral
3998,635769805279248384,positive
